In [20]:
import pandas as pd
import netCDF4 as nc
import numpy as np
import xarray as xr

In [21]:
drifter_2005=nc.Dataset('NOAA_drifter_2005.nc','r')

In [22]:
times_2005=drifter_2005.variables['times'][:]
lons_2005=drifter_2005.variables['lons'][:]
lats_2005=drifter_2005.variables['lats'][:]
drifter_times = pd.DataFrame(times_2005)
drifter_lons = pd.DataFrame(lons_2005)
drifter_lats=pd.DataFrame(lats_2005)
drifter_times=drifter_times*24

In [23]:
for i in range(31):
    var=f"drifter{i+1}"
    globals()[var]=pd.DataFrame({'time':drifter_times.loc[i] , 'lons':drifter_lons.loc[i], 'lats' : drifter_lats.loc[i]}).dropna()

drifter_2005.close()

In [24]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_sea.nc', engine='netcdf4')

for i in range(31):
    name=f"drifter{i+1}"
    if name in globals():
        drifter = globals()[name]
        uo_list=[]
        vo_list=[]
        for j in range(len(drifter)):
            input_time_hours=drifter.loc[j]['time']
            # 시간을 datetime 형식으로 변환0
            base_time = np.datetime64('2005-01-01T00:00:00')
            input_time = base_time + np.timedelta64(int(input_time_hours), 'h')
            input_longitude = drifter.loc[j]['lons']  # 경도 입력
            input_latitude = drifter.loc[j]['lats']   # 위도 입력

            nearest_point = dataset['uo'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            uo_value = nearest_point.item()
            uo_list.append(uo_value)

            nearest_point = dataset['vo'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            vo_value = nearest_point.item()
            vo_list.append(vo_value)

    drifter['uo']=uo_list
    drifter['vo']=vo_list

In [25]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_wind.nc', engine='netcdf4')

for i in range(31):
    name=f"drifter{i+1}"
    if name in globals():
        drifter = globals()[name]
        u10_list=[]
        v10_list=[]
        for j in range(len(drifter)):
            input_time_hours=drifter.loc[j]['time']
            # 시간을 datetime 형식으로 변환0
            base_time = np.datetime64('2005-01-01T00:00:00')
            input_time = base_time + np.timedelta64(int(input_time_hours), 'h')
            input_longitude = drifter.loc[j]['lons']  # 경도 입력
            input_latitude = drifter.loc[j]['lats']   # 위도 입력

            nearest_point = dataset['u10'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            u10_value = nearest_point.item()
            u10_list.append(u10_value)

            nearest_point = dataset['v10'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            v10_value = nearest_point.item()
            v10_list.append(v10_value)

    drifter['u10']=u10_list
    drifter['v10']=v10_list

In [37]:
drifter31

,time,lons,lats,uo,vo,u10,v10,next_time,next_lons,next_lats
0,8346.0,129.369003,35.027000,0.253617,0.248319,6.007233,-3.038177,8352.0,129.335999,34.969002
1,8352.0,129.335999,34.969002,0.219308,0.171529,7.527520,-2.418795,8358.0,129.425995,35.046001
2,8358.0,129.425995,35.046001,0.317718,0.178381,9.893673,-3.561818,8364.0,129.503006,35.067001
3,8364.0,129.503006,35.067001,0.195717,0.151427,8.572718,-3.107539,8370.0,129.617004,35.113998
4,8370.0,129.617004,35.113998,0.213457,0.155442,8.033594,-3.234542,8376.0,129.593002,35.097000
...,...,...,...,...,...,...,...,...,...,...
1160,15306.0,147.319000,44.168999,0.056983,-0.075745,2.859889,2.814660,15312.0,147.348999,44.167999
1161,15312.0,147.348999,44.167999,0.076788,-0.085588,3.303449,1.276216,15318.0,147.307007,44.138000
1162,15318.0,147.307007,44.138000,0.080270,-0.089530,2.458774,1.746053,15324.0,147.250000,44.146999
1163,15324.0,147.250000,44.146999,0.073061,-0.081354,1.912282,3.096562,15330.0,147.322998,44.201000


In [27]:
for i in range(31):
    var = f"drifter{i+1}"
    next_time = globals()[var]['time'][1:]
    next_lons = globals()[var]['lons'][1:]
    next_lats = globals()[var]['lats'][1:]
    next_data = pd.DataFrame({'next_time': next_time, 'next_lons': next_lons, 'next_lats': next_lats})
    next_data = next_data.reset_index()
    globals()[var] = globals()[var].drop(globals()[var].index[-1])
    globals()[var] = pd.concat([globals()[var], next_data],axis=1)
    globals()[var] = globals()[var].drop('index', axis=1).dropna()


In [38]:
import pandas as pd

# 데이터프레임을 담을 빈 리스트 생성
dataframes = []

# 이미 생성된 데이터프레임들을 리스트에 추가
for i in range(1, 32):
    var_name = f"drifter{i}"
    dataframes.append(globals()[var_name])

# 데이터프레임을 연결 (concatenate)
result_df = pd.concat(dataframes, ignore_index=True)

# 결과 확인
print(result_df)

          time        lons       lats        uo        vo       u10       v10  \
0       1230.0  129.311005  34.950001  0.145958  0.086236  6.700244 -0.549905   
1       1236.0  129.401993  35.020000  0.159561  0.139138  3.750556 -1.767175   
2       1242.0  129.514008  35.061001  0.089861  0.055524  4.140956 -1.687066   
3       1248.0  129.524002  35.032001  0.103777  0.132757  1.455666  0.538409   
4       1254.0  129.626007  35.083000  0.272004  0.200787  5.819262  8.927412   
...        ...         ...        ...       ...       ...       ...       ...   
15322  15306.0  147.319000  44.168999  0.056983 -0.075745  2.859889  2.814660   
15323  15312.0  147.348999  44.167999  0.076788 -0.085588  3.303449  1.276216   
15324  15318.0  147.307007  44.138000  0.080270 -0.089530  2.458774  1.746053   
15325  15324.0  147.250000  44.146999  0.073061 -0.081354  1.912282  3.096562   
15326  15330.0  147.322998  44.201000  0.064945 -0.066032  0.093472  3.826448   

       next_time   next_lon

In [57]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_sea.nc', engine='netcdf4')  # 자료 파일 이름을 지정하세요

input_time_hours = float(input("Enter time (hours since 2005-01-01 00:00:00): "))  # 시간 입력 (시간 단위)

# 시간을 datetime 형식으로 변환
base_time = np.datetime64('2005-01-01T00:00:00')
input_time = base_time + np.timedelta64(int(input_time_hours), 'h')

# 사용자로부터 입력받은 경도와 위도
input_longitude = float(input("Enter longitude: "))  # 경도 입력
input_latitude = float(input("Enter latitude: "))    # 위도 입력

# 입력된 시간과 위치에 가장 가까운 자료 포인트 선택
nearest_point = dataset['uo'].sel(
    time=input_time,
    longitude=input_longitude,
    latitude=input_latitude,
    method='nearest'
)

uo_value = nearest_point.item()

nearest_point = dataset['vo'].sel(
    time=input_time,
    longitude=input_longitude,
    latitude=input_latitude,
    method='nearest'
)

vo_value = nearest_point.item()


# 찾은 가장 가까운 포인트의 시간, 경도, 위도
#nearest_time = nearest_point.time.item()
#nearest_longitude = nearest_point.longitude.item()
#nearest_latitude = nearest_point.latitude.item()


vo_value = nearest_point.item()

# 결과 출력

print(f'uo value at nearest point and time: {uo_value} m/s')
print(f'vo value at nearest point and time: {vo_value} m/s')

# 파일을 닫습니다.
dataset.close()

Enter time (hours since 2005-01-01 00:00:00):  4956
Enter longitude:  137.875
Enter latitude:  37.20


uo value at nearest point and time: nan m/s
vo value at nearest point and time: nan m/s
